Training for efficientnet on cropped image patches

In [1]:
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from PIL import Image, ImageOps
import os
import argparse
import time
import os, sys
from time import strftime
from sklearn.metrics import mean_squared_error, accuracy_score, hamming_loss, roc_curve, auc, f1_score
import copy
from torch.utils.data import DataLoader, Dataset
import random
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import PIL.Image as Image
import torch.optim as optim
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torchvision.models as models
from torch.autograd import Variable
import torch.nn as nn
import cv2
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf #2.8.3
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from efficientnet.keras import center_crop_and_resize, preprocess_input

In [ ]:
class data_loader(Sequence):
    """
    Dataset to read image and label for training
    """
    def __init__(self, list_IDs, labels, batch_size=64, dim=(224,224), n_channels=3,
                 n_classes=2, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.asarray(Image.open(ID))[...,:3]
            # Store class
            y[i] = self.labels[ID]

        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
rand_seed = 26700
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)

use_gpu = torch.cuda.is_available()
print('Using GPU: ', use_gpu)

device = torch.device("cuda:0")    

from os import listdir
from os.path import isfile, join

In [ ]:
train_pos = r"./crops_3/training_pos" #"C:\Users\emm75\Documents\BMI_771\BMI771_Project\crops_1\training_pos"
train_neg = r"./crops_3/training_neg" #"C:\Users\emm75\Documents\BMI_771\BMI771_Project\crops_1\training_neg"
val_pos = r"./crops_3/val_pos" #"C:\Users\emm75\Documents\BMI_771\BMI771_Project\crops_1\val_pos"
val_neg = r"./crops_3/val_neg" #"C:\Users\emm75\Documents\BMI_771\BMI771_Project\crops_1\val_neg"

img_train_pos = [join(train_pos, f) for f in listdir(train_pos) if isfile(join(train_pos, f))]
img_train_neg = [join(train_neg, f) for f in listdir(train_neg) if isfile(join(train_neg, f))]
img_val_pos = [join(val_pos, f) for f in listdir(val_pos) if isfile(join(val_pos, f))]
img_val_neg = [join(val_neg, f) for f in listdir(val_neg) if isfile(join(val_neg, f))]

In [ ]:
IMG_SIZE = 224
NUM_CLASSES = 2 
batch = 64

features = {'dim': (224,224),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

label_gen = {}
i_pos = img_train_pos + img_val_pos
for i in i_pos:
    label_gen[i] = 1
i_neg = img_train_neg + img_val_neg
for j in (i_neg):
    label_gen[j] = 0

img_gen = {'train': [], 'val': []}
g_pos = img_train_pos + img_train_neg
for k in g_pos:
    img_gen['train'] = img_gen['train'] + [k]
g_neg = img_val_pos + img_val_neg
for l in (g_neg):
    img_gen['test'] = img_gen['test'] + [l]

print(img_gen['train'])

In [ ]:
train_set = data_loader(img_gen['train'], label_gen, **features)
val_set = data_loader(img_gen['test'], label_gen, **features)

In [ ]:
layer_train = Sequential(
    [
        layers.RandomFlip(mode="horizontal"),
        layers.RandomFlip(mode="vertical"),
    ],
    name = 'layer_train',
)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with tf.distribute.MirroredStrategy.scope():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    trained = layer_train(inputs)
    outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(trained)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.summary()

In [ ]:
epochs = 40

hist = model.fit(train_set, epochs=epochs, validation_data=val_set, verbose=2)

In [ ]:
pred_pos = r"./crops_1/val_pos"
pred_neg = r"./crops_1/val_pos"

img_pred_pos = [join(pred_pos, f) for f in listdir(pred_pos) if isfile(join(pred_pos, f))]
img_pred_neg = [join(pred_neg, f) for f in listdir(pred_neg) if isfile(join(pred_neg, f))]

lab_gen = {}
pred_gen = {'predict': []}
for i in img_pred_pos:
    lab_gen[i] = 1 
    pred_gen['predict'] = pred_gen['predict'] + [i]
for j in (img_pred_neg):
    lab_gen[j] = 0
    pred_gen['predict'] = pred_gen['predict'] + [j]

pred_set = data_loader(pred_gen['predict'], lab_gen, **features)

In [ ]:
hist.history

In [ ]:
model.evaluate(pred_set, batch_size=64)

In [ ]:
matrix = model.predict(pred_set)
matrix

In [ ]:
type(matrix)
np.save("large_2_40ep",matrix)
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")